# CSE547 - Colab 5
## PageRank

Adapted from Stanford's CS 246

### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

If you want to explore NetworkX more, consider following [this tutorial](https://networkx.org/documentation/stable/tutorial.html), although it isn't necessary for this Colab.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [ ]:
!pip install networkx==2.6.3
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [ ]:
print("Graph nodes = ", G.number_of_nodes())
print("Graph edges = ", G.number_of_edges())

Graph nodes =  281903
Graph edges =  2312497


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

Functions you'll probably need to use: 
* [nx.weakly_connected_components](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.weakly_connected_components.html)
* [G.subgraph](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html)

In [ ]:
# YOUR CODE HERE
lst_max_weakly_connected_components = max(nx.weakly_connected_components(G))
len(lst_max_weakly_connected_components)

255265

In [ ]:
lst_max_weakly_connected_components

In [ ]:
G.remove_nodes_from(set(G.nodes()) - lst_max_weakly_connected_components)
#print(nx.info(G))

Compute the PageRank vector, using the default parameters in NetworkX: [pagerank](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

Functions you'll probably need to use: 
* [nx.pagerank](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.)

In [ ]:
# YOUR CODE HERE
#import scipy
pageRank = nx.pagerank(G)
len(pageRank)

255265

In [ ]:
degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
dmax = max(degree_sequence)
G.out_degree

In [ ]:
degree_sequence = sorted((d for n, d in G.out_degree), reverse=True)
np.mean(degree_sequence)

8.753930229369479

In [ ]:
pageRank[max(pageRank, key=pageRank.get)]

0.011051585882434985

In [ ]:
np.mean(list(pageRank.values()))

3.917497502595348e-06

In [ ]:
pageRank['89073']

0.011051585882434985

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using for both methods ```seed = 1```, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.

Functions you'll probably need to use: 
* [nx.pagerank](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.)
* [nx.fast_gnp_random_graph](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.fast_gnp_random_graph.html)
* [nx.barabasi_albert_graph](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.barabasi_albert_graph.html)


In [ ]:

# YOUR CODE HERE
nb_nodes = G.number_of_nodes()
nb_edges = G.number_of_edges()

randomGraph = nx.fast_gnp_random_graph(n=nb_nodes, p = 0.00008, seed=1)
barabasiAlbertGraph = nx.barabasi_albert_graph(n=nb_nodes, m=nb_edges// nb_nodes+1, seed=1)

In [ ]:
print(nx.info(randomGraph))
pageRankRandomGraph = nx.pagerank(randomGraph)
len(pageRankRandomGraph)

Graph with 255265 nodes and 2606386 edges


255265

In [ ]:
max(pageRankRandomGraph, key=pageRankRandomGraph.get)

150787

Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort the components of each vector by value**, and use cosine similarity as similarity measure. 

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [ ]:

# YOUR CODE HERE
pageRankRandomGraph = nx.pagerank(randomGraph)
len(pageRankRandomGraph)
     
pageRankBarabasiAlbertGraph = nx.pagerank(barabasiAlbertGraph)
len(pageRankBarabasiAlbertGraph)

255265

In [ ]:
pageRankBarabasiAlbertGraph[max(pageRankBarabasiAlbertGraph, key=pageRankBarabasiAlbertGraph.get)]

0.00041689766748208805

In [ ]:

sortedPageRank = np.array(sorted(pageRank.values()))
sortedPageRankRandomGraph = np.array(sorted(pageRankRandomGraph.values()))
sortedPageRankBarabasiAlbertGraph = np.array(sorted(pageRankBarabasiAlbertGraph.values()))

In [ ]:
from scipy import spatial

print(float(spatial.distance.cosine(sortedPageRank, sortedPageRankRandomGraph)))
print(float(spatial.distance.cosine(sortedPageRank, sortedPageRankBarabasiAlbertGraph)))
a = list(sorted(pageRank.values()))
b = list(sorted(pageRankBarabasiAlbertGraph.values()))

dot(a, b)/(norm(a)*norm(b))

0.896044352960356
0.3511326339032764


0.648867366096724

In [ ]:
from numpy import dot
from numpy.linalg import norm




Gnp = nx.fast_gnp_random_graph(255265, 0.00008, seed = 1)

print(nx.info(Gnp))

Gba = nx.barabasi_albert_graph(255265, 9, seed = 1)

print(nx.info(Gba))
pr_ba = nx.pagerank(Gba)
pr_np = nx.pagerank(Gnp)


Graph with 255265 nodes and 2606386 edges
Graph with 255265 nodes and 2297304 edges


NameError: ignored

In [ ]:
Gc = max(nx.weakly_connected_components(G), key=len)
Gc = G.subgraph(Gc).copy()

pr = nx.pagerank(Gc)
a = list(sorted(pr.values()))
b = list(sorted(pr_np.values()))

dot(a, b)/(norm(a)*norm(b))

0.10395564703964398

In [ ]:
dot(a, b)/(norm(a)*norm(b))

In [ ]:
from numpy import dot
from numpy.linalg import norm

a = list(sorted(pr.values()))
b = list(sorted(pr_ba.values()))

dot(a, b)/(norm(a)*norm(b))

0.648867366096724

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!